In [1]:
import numpy as np, pandas as pd
import sympy as sym
from sympy import symbols, Matrix, solve, simplify
import math

In [2]:
#### initialize unknown variable symbols 

i_S1, i_S2, i_S3 = symbols('i_S1 i_S2 i_S3')
i_D1, i_D2, i_D3 = symbols('i_D1 i_D2 i_D3')

In [3]:
unknowns = [i_S1, i_S2, i_S3,
           i_D1, i_D2, i_D3]
len(unknowns)

6

In [4]:
### initialize known variable symbols

p_4, p_3, p_2, p_1 = symbols('p_4 p_3 p_2 p_1')
i1, i2, i3 = symbols('i1 i2 i3')
di_1, di_2, di_3, di_4 = symbols('di_1 di_2 di_3 di_4')
dr_1, dr_2, dr_3, dr_4 = symbols('dr_1 dr_2 dr_3 dr_4')
b_D3, b_D2, b_D1 = symbols('b_D3 b_D2 b_D1')
m_D3, m_D2, m_D1 = symbols('m_D3 m_D2 m_D1')
r_D3, r_D2, r_D1tx, r_D1ux = symbols('r_D3 r_D2 r_D1tx r_D1ux')

In [5]:
#[{unknown:coefficient, unknown:coefficient}, right hand side]

eq1 = [{i_S1:1, i_D1:1}, i1 * p_2]
eq2 = [{i_S2:1, i_D2:1}, i2 * p_3]
eq3 = [{i_S3:1, i_D3:1}, i3 * p_4]
eq4 = [{i_D2:1, i_D3:-1}, di_3 + b_D3 + r_D2 + r_D1tx - r_D3 - dr_3 - m_D3]
eq5 = [{i_D1:1, i_D2:-1}, di_1 + r_D1ux + b_D2 - r_D2 - dr_2 - m_D2]
eq6 = [{i_D1:1}, dr_1 + m_D1 + r_D1tx + r_D1ux - di_1 - b_D1]

In [6]:
eqs = [eq1,eq2,eq3,eq4,eq5,eq6]
len(eqs)

6

In [7]:
def build_matrix(eqns, unknowns):
    """
    INPUT
    ----
    eqns: a list of equations. These are of the form:
          [{x_i:a, x_j:b}, y] for the equation:
          ax_i + bx_j = y
    unknowns: a list of sympy unknowns
    ----
    OUTPUT
    ----
    A:  a matrix containing the coefficients of left-hand side of all eq in eqns.
        - nrows = number of equations
        - rcols = number of unknowns
    b: an nx1 matrix containing the right-hand side of all the eqns
    x: a sympy matrix of the unknowns
    
    Note that Ax = b
    """
    n_eqns = len(eqns)
    n_unknowns = len(unknowns)

    # frame for matrix/LHS equation coefficients.
    # nrows = n_eqns, ncols = n_unknowns
    A = pd.DataFrame(
        index = range(n_eqns),
        columns = unknowns,
        data = np.zeros([n_eqns,n_unknowns])
    )

    # frame for RHS of equations
    b = pd.DataFrame(index = range(n_eqns), columns = ['val'])
    
    # define helper fn
    def add_eq(terms, y, i):
        """
        INPUT
        ----
        To input i^(th) equation y = ax_i + bx_j, add_eq wants:
            - terms = {x_i:a, x_j:b}
            - y = y
            - i
        ----
        FUNCTION
        ----
        - Adds coefficients to matrix A.
        - Adds y to row i of vector b
        ----
        EXAMPLE
        ----
        For y = ax_i + bx_j:
        - adds "a" to column "x_i" row i of A
        - adds "b" to column "x_j" row i of A
        - adds "y" to row i of b
        """
        for x in terms.keys():
            A[x][i] = terms[x]
        b.iloc[i] = y

    # populate LHS/RHS
    i = 0
    for eq in eqns:
        add_eq(terms=eq[0], y=eq[1], i=i)
        i += 1

    # convert to sympy matrices
    A = sym.Matrix(A)
    b = sym.Matrix(b)
    x = sym.Matrix(unknowns) #vars to solve for

    return A, x, b

In [8]:
A1, x1, b1 = build_matrix(eqs, unknowns)

In [9]:
A1

Matrix([
[1.0, 0.0, 0.0, 1.0,  0.0,  0.0],
[0.0, 1.0, 0.0, 0.0,  1.0,  0.0],
[0.0, 0.0, 1.0, 0.0,  0.0,  1.0],
[0.0, 0.0, 0.0, 0.0,  1.0, -1.0],
[0.0, 0.0, 0.0, 1.0, -1.0,  0.0],
[0.0, 0.0, 0.0, 1.0,  0.0,  0.0]])

In [10]:
A1.shape

(6, 6)

In [11]:
x1

Matrix([
[i_S1],
[i_S2],
[i_S3],
[i_D1],
[i_D2],
[i_D3]])

In [12]:
x1.shape

(6, 1)

In [13]:
b1

Matrix([
[                                          i1*p_2],
[                                          i2*p_3],
[                                          i3*p_4],
[b_D3 + di_3 - dr_3 - m_D3 + r_D1tx + r_D2 - r_D3],
[       b_D2 + di_1 - dr_2 - m_D2 + r_D1ux - r_D2],
[    -b_D1 - di_1 + dr_1 + m_D1 + r_D1tx + r_D1ux]])

In [14]:
b1.shape

(6, 1)

In [15]:
result = sym.solve(A1 * x1 - b1, x1)#, dict=True, set=True)
result

{i_S1: b_D1 + di_1 - dr_1 + i1*p_2 - m_D1 - r_D1tx - r_D1ux,
 i_S2: b_D1 + b_D2 + 2.0*di_1 - dr_1 - dr_2 + i2*p_3 - m_D1 - m_D2 - r_D1tx - r_D2,
 i_S3: b_D1 + b_D2 + b_D3 + 2.0*di_1 + di_3 - dr_1 - dr_2 - dr_3 + i3*p_4 - m_D1 - m_D2 - m_D3 - r_D3,
 i_D1: -b_D1 - di_1 + dr_1 + m_D1 + r_D1tx + r_D1ux,
 i_D2: -b_D1 - b_D2 - 2.0*di_1 + dr_1 + dr_2 + m_D1 + m_D2 + r_D1tx + r_D2,
 i_D3: -b_D1 - b_D2 - b_D3 - 2.0*di_1 - di_3 + dr_1 + dr_2 + dr_3 + m_D1 + m_D2 + m_D3 + r_D3}